# NWS Warning Polygons

In [1]:
#Import the necessary packages
import numpy as np
import datetime as dt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import noaa_sdk as noaa_sdk
import pandas as pd
import shapely as shapely
from   metpy.plots       import  USCOUNTIES



import timezonefinder    as tzf
import pytz              as pytz


import geopandas as gp

length = 11
height =  6

dpi = 100
length = 19
height = 10.36538462
font_fac = 1.923076923


In [2]:

####################################################
####################################################
####################################################
#
# Mines Colors and Fonts
#

Mines_Blue = "#002554"


plt.rcParams.update({'text.color'      : Mines_Blue,
                     'axes.labelcolor' : Mines_Blue,
					 'axes.edgecolor'  : Mines_Blue,
					 'xtick.color'     : Mines_Blue,
					 'ytick.color'     : Mines_Blue})


#
####################################################
####################################################
####################################################



In [3]:
proj_data_text = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs'

gif_file_name = "./graphics_files/NWS_Warnings.png"
xls_file_name = "./graphics_files/NWS_Warnings.xlsx"



myproj = ccrs.AlbersEqualArea(central_longitude=-96, 
                              central_latitude=37.5, 
                              false_easting=0.0, 
                              false_northing=0.0, 
                              standard_parallels=(29.5, 45.5))



tz='America/Denver'

time_utc = dt.datetime.utcnow()
valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H%M %Z")
local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H%M %Z")

print(valid_time)
print(local_time)

2023-05-14 2139 UTC
2023-05-14 1539 MDT


In [4]:
# Open the NWS API in python to get the active alerts
n = noaa_sdk.noaa.NOAA()
alerts = n.active_alerts()


In [5]:
#injest //shapefiles/CONUS_UGC_Zones/


UGC_Zones_Shapefile    = gp.read_file('./shapefiles/CONUS_UGC_Zones/CONUS_UGC_Zones.shp')
UGC_Counties_Shapefile = gp.read_file('./shapefiles/CONUS_UGC_Counties/CONUS_UGC_Counties.shp')
 
UGC_Shapefile = pd.concat([UGC_Zones_Shapefile, 
                           UGC_Counties_Shapefile]).drop(["UGC_Temp",
                                                          "Shape_Leng",
                                                          "Shape_Le_1",
                                                          "Shape_Area",
                                                          "STATE",
                                                          "CWA",
                                                          "COUNTYNAME",
                                                          "FIPS",
                                                          "TIME_ZONE",
                                                          "FE_AREA"],
                                                          axis = 'columns').reset_index(drop=True)  
    
UGC_Zone_County_List = UGC_Shapefile['UGC'].to_list()



In [6]:
# priority warnings table
warning_priority_table = pd.read_csv("./warning_table_sorted.csv")

warning_priority_table = warning_priority_table.rename(columns={"hdln": "event"})



In [7]:

current_warnings = pd.DataFrame(columns = ['event',
                                           'UGC',
                                           'coverage',
                                           'message'])



i = 0



for alert in alerts['features']:
    event = alert['properties']['event']
    if (event != "Test Message"): 
        
        try:

            ugc_codes = alert['properties']['geocode']['UGC']
            message   = alert['properties']['@id']

            for ugc_code in ugc_codes:
                if ugc_code in UGC_Zone_County_List: 
                    if (ugc_code[2] == 'C'):
                        coverage = "County"
                    else:
                        coverage = "Zones"

                    deleteme = pd.DataFrame([[event,
                              ugc_code,
                              coverage,
                              message]], 
                              columns = ['event',
                                         'UGC',
                                         'coverage',
                                         'message'])
                    current_warnings = pd.concat([current_warnings, 
                                                  deleteme]) 
                    i = i + 1
        except KeyError:
            print("poopie:")
            print(event, ugc_code)
    
current_warnings = current_warnings.merge(warning_priority_table, how='left', on='event')
current_warnings = UGC_Shapefile.merge(current_warnings, how='right', on='UGC')
current_warnings = current_warnings.sort_values("Rank", ascending=False)


if (current_warnings['color'].isnull().values.any()):
    print("Missing Colors Found")
    locs_missing = current_warnings.index[current_warnings['color'].isnull()].tolist()
    print(current_warnings.loc[locs_missing])

    print("replacing Nones Complete")
  
    for loc_missing in locs_missing:
        current_warnings.loc[loc_missing,"color"] = 'red'
        print(current_warnings.loc[locs_missing])
print("replaced Nones Complete")





warning_color_table = current_warnings[["event","color"]].drop_duplicates()


current_warnings.drop(["geometry"], axis="columns").to_excel(xls_file_name)

print("done: ",i,"rows; ",len(warning_color_table),"event types")
print()

replaced Nones Complete
done:  1118 rows;  18 event types



In [8]:
# check for missing colors.

if (warning_color_table['color'].isnull().values.any()):
    print("Missing Colors Found")
    loc_missing = warning_color_table.index[warning_color_table['color'].isnull()].tolist()
    print(loc_missing)
    print(warning_color_table.loc[loc_missing])
    warning_color_table.loc[loc_missing,"color"] = '#ffffff'
    
print("replaced")
print(warning_color_table)

replaced
                            event           color
993            Hydrologic Outlook         #00ff7f
750      Marine Weather Statement        moccasin
911     Special Weather Statement        moccasin
334       Beach Hazards Statement       turquoise
1029        Rip Current Statement            aqua
1253               Flood Advisory         #00ff7f
369                Frost Advisory  cornflowerblue
1120                Heat Advisory         #ff7f4e
1063           Lake Wind Advisory             tan
258          Small Craft Advisory         thistle
1153                  Flood Watch        seagreen
91              Flash Flood Watch       limegreen
304            Fire Weather Watch     navajowhite
253     Severe Thunderstorm Watch   palevioletred
382                Freeze Warning            cyan
71            Flash Flood Warning         darkred
1085                Flood Warning           green
19    Severe Thunderstorm Warning          orange


In [9]:
legend_color_table = warning_color_table.values.tolist()
print(legend_color_table)
legend_color_table = []

for row in warning_color_table.iterrows():
    mypatch = [mpatches.Patch(color=row[1][1], label=row[1][0])]
    legend_color_table = legend_color_table + mypatch
    
    
  

[['Hydrologic Outlook', '#00ff7f'], ['Marine Weather Statement', 'moccasin'], ['Special Weather Statement', 'moccasin'], ['Beach Hazards Statement', 'turquoise'], ['Rip Current Statement', 'aqua'], ['Flood Advisory', '#00ff7f'], ['Frost Advisory', 'cornflowerblue'], ['Heat Advisory', '#ff7f4e'], ['Lake Wind Advisory', 'tan'], ['Small Craft Advisory', 'thistle'], ['Flood Watch', 'seagreen'], ['Flash Flood Watch', 'limegreen'], ['Fire Weather Watch', 'navajowhite'], ['Severe Thunderstorm Watch', 'palevioletred'], ['Freeze Warning', 'cyan'], ['Flash Flood Warning', 'darkred'], ['Flood Warning', 'green'], ['Severe Thunderstorm Warning', 'orange']]


In [10]:
print(warning_color_table)


                            event           color
993            Hydrologic Outlook         #00ff7f
750      Marine Weather Statement        moccasin
911     Special Weather Statement        moccasin
334       Beach Hazards Statement       turquoise
1029        Rip Current Statement            aqua
1253               Flood Advisory         #00ff7f
369                Frost Advisory  cornflowerblue
1120                Heat Advisory         #ff7f4e
1063           Lake Wind Advisory             tan
258          Small Craft Advisory         thistle
1153                  Flood Watch        seagreen
91              Flash Flood Watch       limegreen
304            Fire Weather Watch     navajowhite
253     Severe Thunderstorm Watch   palevioletred
382                Freeze Warning            cyan
71            Flash Flood Warning         darkred
1085                Flood Warning           green
19    Severe Thunderstorm Warning          orange


In [11]:
bbox=[-120,-73,22.5,50]

xxx = [-120.0, -73.0]
yyy = [  22.5,  50.0]


fig = plt.figure(figsize   = [length, height], dpi = dpi,
                 facecolor = 'white')

ax = fig.add_subplot(1, 1, 1, 
                     projection=myproj)

plt.suptitle("NWS Watches and Warnings",
             fontsize = 20*font_fac, 
             color    = Mines_Blue)
ax.set_extent(bbox)
ax.set_title(valid_time + "  (" + local_time+")",
             fontsize=15*font_fac, 
             color=Mines_Blue)

current_warnings.plot(ax = ax,aspect='equal',
                      facecolor=current_warnings["color"],
                      transform=ccrs.PlateCarree(),
                      edgecolor='None', linewidth=0)

ax.add_feature(cfeature.COASTLINE.with_scale('50m'), 
               linewidth = 0.5,
			  edgecolor=Mines_Blue)
ax.add_feature(cfeature.STATES.with_scale('50m'),    
               linewidth = 0.25, 
               edgecolor = Mines_Blue)
ax.add_feature(cfeature.LAKES.with_scale('50m'),   
               linewidth = 0.5,
               facecolor = "none", 
               edgecolor = Mines_Blue)
ax.add_feature(feature    = USCOUNTIES, 
                   linewidths = 0.1,
                   edgecolor  = Mines_Blue,
                   facecolor  = 'none')

ax.set_frame_on(False)

plt.subplots_adjust(left   = 0.01, 
                    right  = 0.75, 
                    top    = 0.91, 
                    bottom = 0.01)

labelspacing = 0.1
fig.legend(handles  = legend_color_table, 
           loc      = 'right',
           frameon  = False,fontsize=18,
           labelspacing = labelspacing,
			  labelcolor = Mines_Blue)




#########################################
#
# Insert a Clock
#

axins = fig.add_axes(rect     =    [0, # 0.015,
                                    1-0.12, #0.015,
                                    0.12*8/9,
                                    0.12],
                      projection  =  "polar")

time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

hour   = time_for_clock.hour
minute = time_for_clock.minute
second = time_for_clock.second

circle_theta  = np.deg2rad(np.arange(0,360,0.01))
circle_radius = circle_theta * 0 + 1

if (hour > 12) :
    hour = hour - 12

angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)

print(time_for_clock)
print(hour,   np.rad2deg(angles_h))
print(minute, np.rad2deg(angles_m))


plt.setp(axins.get_yticklabels(), visible=False)
plt.setp(axins.get_xticklabels(), visible=False)
axins.spines['polar'].set_visible(False)
axins.set_ylim(0,1)
axins.set_theta_zero_location('N')
axins.set_theta_direction(-1)
axins.set_facecolor("white")
axins.grid(False)

axins.plot([angles_h,angles_h], [0,0.6], color=Mines_Blue, linewidth=1.5*font_fac)
axins.plot([angles_m,angles_m], [0,0.95], color=Mines_Blue, linewidth=1.5*font_fac)
axins.plot(circle_theta, circle_radius, color="darkgrey", linewidth=1*font_fac)


#
#########################################
        
        
plt.savefig(gif_file_name, dpi = dpi,
                        facecolor   = 'white', 
                        transparent =   False)


plt.close()


print("done")

15:39:15.986150
3 109.53978873577297
39 235.5


/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


done


In [12]:
import matplotlib as mpl


In [13]:
mpl.rcParams['lines.linewidth']

1.5

In [14]:
mpl.rcParams['legend.fontsize']

'medium'

In [15]:
mpl.rcParams['.fontsize']

KeyError: '.fontsize'